<a href="https://colab.research.google.com/github/sukantjain/Python/blob/main/NetCDF_to_CSV.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# This is script is prepared to extract to data from NetCDF (.nc) files and save it into CSV format

In [ ]:
import xarray
import pandas as pd
import numpy as np

In [ ]:
# Provide the path where all NetCDF (.nc) files are stored
# You can also upload the files into GDrive also
netCDF_path = '/content'

dsmerged = xarray.open_mfdataset(netCDF_path+'/*.nc')
dsmerged

In [ ]:
# Provide variable name as displayed in the result of above cell

var_name = 'precip'
lat_name = 'latitude'
long_name = 'longitude'
time_name = 'time'

arr_lat=np.array(dsmerged[lat_name])
arr_long=np.array(dsmerged[long_name])
arr_time=np.array(dsmerged[time_name])

variable = dsmerged[var_name]
variable[-1,:,:].plot(figsize = (10,5))


In [ ]:
# Function to extract the data
def get_closest_rain_data(user_lat, user_long, arr_lat, arr_long, pcp2):
    # Find the closest latitude and longitude indices
    closest_index_lat = np.argmin(np.abs(arr_lat - user_lat))
    closest_index_long = np.argmin(np.abs(arr_long - user_long))

    # Get the closest latitude and longitude values
    closest_lat = arr_lat[closest_index_lat]
    closest_long = arr_long[closest_index_long]

    # Print the closest lat/long and their indices
    print("Closest latitude:", closest_lat, "at index:", closest_index_lat)
    print("Closest longitude:", closest_long, "at index:", closest_index_long)

    # Get the rainfall data for the closest location
    arr_rain = pcp2[:, closest_index_lat, closest_index_long]

    # Create the name string
    name = f"{closest_lat}_{closest_long}"

    return arr_rain, name

In [ ]:
# To download data of a single location into csv
#  Please provide lat and long

# 28.679, 77.069 Delhi
user_lat = 28.679
user_long = 77.069


arr_rain, name = get_closest_rain_data(user_lat, user_long, arr_lat, arr_long, variable)
df2 = pd.DataFrame({name: arr_rain}, index=arr_time)

# Uncomment this line to download the data into csv
# df2.to_csv(var_name + '_' +name+'.csv')
df2

In [ ]:
# To download data of a bounding box into csv
#  Please provide upper and lower lat and long

upper_lat = 35
lower_lat = 30
upper_long = 80
lower_long = 73

def slicing_variable(variable, lat_name, lon_name, lower_lat, upper_lat, lower_long, upper_long):
    variable2 = variable.sel({lat_name: slice(lower_lat, upper_lat), long_name: slice(lower_long, upper_long)})
    # print (variable2)
    # variable2[-1, :, :].plot()
    return variable2

variable2 = slicing_variable(variable, lat_name, long_name, lower_lat, upper_lat, lower_long, upper_long)
if variable2.size == 0:
    variable2 = slicing_variable(variable, lat_name, long_name, upper_lat, lower_lat, lower_long, upper_long)

variable2[-1, :, :].plot()

In [ ]:
varList = []
nameList = []

arr_lat=np.array(variable2[lat_name])
arr_long=np.array(variable2[long_name])
arr_time=np.array(variable2[time_name])

for latt in arr_lat:
  for longg in arr_long:
    print (latt, longg)
    arr_var, name = get_closest_rain_data(latt, longg, arr_lat, arr_long, variable2)
    varList.append(arr_var)
    nameList.append(name)
    # print (arr_var)


data_dict = {name: array for name, array in zip(nameList, varList)}

df3 = pd.DataFrame(data_dict, index=arr_time)

# Uncomment this line to download the data into csv
# df3.to_csv(var_name+'_All.csv')
df3